# Fun with Astrometry

**LSST-DA Data Science Fellowship Program**

*by Alex Drlica-Wagner (2025-09-15)*

In this notebook we explore a few features related to astrometry and proper motion. I've attempted to follow the mathematical notation used in the lecture notes, though you may also want to look at Gary Bernstein's notes from DSFP Session 11.

*Acknowledgements: This notebook draws inspiration from previous DSFP notebooks from Gary Bernstein and Colin Slater.*

In [ ]:
# Standard imports
import numpy as np
import scipy.stats
import pylab as plt
import pandas as pd

## Exercise 1: Centroid Fitting


### The 1D Case

In the lecture notes we asserted several statistical properties that we will now verify. First, let's verify that in the trivial, unbinned case, the uncertainty on the mean decreases as $1/\sqrt{N}$. We will do this by sampling the standard normal distribution (our "PSF") multiple times with different numbers of samples ("photons").

In [ ]:
# Draw the 1D locations "photons" from a "PSF"

# Use scipy.stats to create a Gaussian probability density function (pdf) to represent our PSF.
# (Aside: the random variable classes in scipy.stats are very useful for quick simulations)
mu,sigma = 0, 1
pdf = scipy.stats.norm(mu,sigma)

# Measure the mean calculated from 1000 trials at each number of photons
ntrials = 1000
nphotons = [1, 10, 100, 1000, 10000]
unbinned_means = []
for i in nphotons:
    means = []
    for j in range(ntrials):
        # Draw "i" randome samples from the pdf
        x = pdf.rvs(size=i)
        # Calculate the mean and add it to our list of means
        means.append(np.mean(x))
    unbinned_means.append( [i, means] )

# Note that an equivalent (and faster) way to do this is
unbinned_means = [ (i, np.mean(pdf.rvs(size=(i, ntrials)), axis=0)) for i in nphotons]

In [ ]:
# Plot how the distribution of means changes with photon number
plt.figure()
for n, mean in unbinned_means:
    plt.hist(mean, bins=np.linspace(-1, 1, 100), label=f"Number of Photons: {n}")
plt.xlabel("Unbinned Mean")
plt.ylabel("Number of Trials")
plt.legend()

In [ ]:
std = np.array([np.std(means) for nph, means in unbinned_means])
plt.scatter(nphotons, std, label="Measurements")
plt.plot(nphotons, sigma/np.sqrt(nphotons), '--', color='gray', label=r"$\sigma/\sqrt{N}$")
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.legend()
plt.ylabel("Scatter in the Mean")
plt.xlabel("Number of Photons")

The above example demonstrates that our statistical assumptions worked for continuous, unpixelized samples from a distribution. Now we want to check that the same statistical properties hold for a pixelized distribution.

<span><span style="color:blue;font-weight: bold;">Exercise:</span>  Use `np.histogram` to bin the "photons" into "pixels". Verify that our statistical assumption holds for 1D pixelated data when the mean is calculated as $$\bar{x} = \frac{ \sum_{x} x I_x } {\sum_{x} I_x},$$ where $I_x$ is the intesity (counts) in the pixel located at coordinate $x$.

In [ ]:
# Draw the 1D locations "photons" from a "PSF"
mu,sigma = 0, 1
pdf = scipy.stats.norm(mu,sigma)

# Configure the bins ("pixels")
bins = np.linspace(-10,10,101)
centers = (bins[1:] + bins[:-1])/2.

# Create sources with different numbers of photons
nphotons = [1, 10, 100, 1000, 10000]
# Number of realizations of each source
ntrials = 1000

In [ ]:
# Your code here...




### The 2D Case

Now, let's try the 2D example. First we'll start by defining our 2D point source as a multivariate normal distribution centered at $x,y = (2,0)$ with $\sigma_x = \sigma_y = 1$. (We choose to offset the source and make our image rectangular to help double check that our array indexing is correct.)

In [ ]:
# Create the 2d normal distribution

# The mean of the distribution (offset to help verify indexing)
xtrue,ytrue = 2, 0
mu = np.array([xtrue,ytrue])
# The spread (now a covariance array)
sigma = 1
cov = np.array([ [sigma,0], [0,sigma] ])
# Create the multivariate normal "PSF" 
pdf = scipy.stats.multivariate_normal(mean=mu,cov=cov)

# Create the pixel edges and centers
# (Rectangular image to help verify indexing)
bins = xbins, ybins = [np.linspace(-10,10,101), np.linspace(-15,15,151)]
centers = xcenters, ycenters = [(bins[0][1:] + bins[0][:-1])/2., (bins[1][1:] + bins[1][:-1])/2.]

# x,y coordinates for the center of each pixel
# Note the convention of numpy is array[row, column]...
# The indexing='ij' is a "shortcut" that I "discovered". Does it work? I don't know...
xx,yy = np.meshgrid(xcenters, ycenters, indexing='ij')

# Sanity check for x,y coordinates...
fig,ax = plt.subplots(1,2,figsize=(10,5))
ax[0].pcolormesh(xx,yy,xx)
ax[1].pcolormesh(xx,yy,yy)

<span><span style="color:blue;font-weight: bold;">Exercise:</span> Generate 1000 photons from our 2D representation of a "PSF". Plot the distribution as a scatter plot. Use `np.histogram2d` to bin the photons into pixels. Call the counts in each pixel `Ixy` (the intensity).  Use `pcolormesh` to plot `Ixy` to show what the image produced from a pixelated detector would look like. (I used `plt.pcolormesh(xx,yy,Ixy)`, but any combination that has the correct x,y orientation should work.)

In [ ]:
# Code for your answer here...




<span><span style="color:blue;font-weight: bold;">Exercise:</span> Calculate the 2D centroid of your pixelated distribution, $(\bar{x}, \bar{y})$, using the counts in each pixel, $I(x,y)$ from the formulae

$$\bar{x} = \frac{\sum_{xy} x I_{xy}}{\sum_{xy} I_{xy}},~~ \bar{y} = \frac{\sum_{xy} y I_{xy}}{\sum_{xy} I_{xy}}$$

Plot the centroid on top of your pixelated image. How does it look?

In [ ]:
# Your answer below.




### Adding noise

Now we are going to add some random noise to the counts in each pixel. This is akin to the noise that would be left after subtracting the sky background. For the purposes of this exercise, assume that the noise is Gaussian: $\mathcal{N}(\mu, \sigma)$. Note that the noise after background subtraction will fluctuate both positive and negative.

In [ ]:
# Generate random Gaussian noise left over after sky subtraction.
sky_mu, sky_sigma = 0, 3
sky_noise = scipy.stats.norm(sky_mu, sky_sigma).rvs(size=Ixy.shape)

fig = plt.figure()
plt.pcolormesh(xx, yy, sky_noise)
plt.colorbar(label="Counts")

<span><span style="color:blue;font-weight: bold;">Question:</span> Add the sky noise to your pixelated counts. Plot the image of your source + noise and calculate the centroid $(\bar{x}, \bar{y})$ following the same procedure above. Do this several times for several realizations of the noise. What happens and why?

In [ ]:
# Your code here...




### Windowed Centroiding

As discussed in the lecture, the common way to perform centroiding in the background-dominated regime is to apply an aperture (or window) filter function. We will define two simple top-hat windows below. Applying these windows will basically amount to restricting our centroid measurement to only use the pixels around the source.

In [ ]:
# Setup the top-hat window array

def square_window(x0, y0, half_width):
    """ A square top-hat window.
    
    Parameters
    ----------
    x0, y0 : center of the window
    half_width : half of the width of the square
    
    Returns
    -------
    window : the window array
    """
    window = np.zeros_like(Ixy)
    window  = (np.abs(xx - x0) < half_width)
    window &= (np.abs(yy - y0) < half_width)
    return window
    
def circular_window(x0, y0, radius):
    """ A circular top-hat window.

    Parameters
    ----------
    x0, y0 : center of the window
    radius : radius of the circle.

    Returns
    -------
    window : the window array
    """
    window = np.zeros_like(Ixy)
    window  = np.sqrt( (xx - x0)**2 + (yy - y0)**2) < radius
    return window

In order to apply the filter, we need to make an initial guess at where the window should be centered. We've just seen that our centroid estimate can be way off, so we need another estimate. We could use the true center of the object, but that is cheating. Let's try centering the window on the "peak", i.e., the pixel with the maximum number of counts (this has worked in most realizations of the noise that I have seen). In a more realistic example, you would have already run source detection, which will have given you a rough estimate for where the source is located.

<span><span style="color:blue;font-weight: bold;">Exercise:</span> Find the x,y coordinates of the pixel with the maximum value. Plot each of the two window functions centered on that pixel with a half-width/radius of `3*sigma`, where `sigma` is the effective "PSF" width that we to simulate our source (our results won't be too sensitive to our choice, and we can assume that we have measured this "PSF" from bright stars...).

In [ ]:
# Your code here...




<span><span style="color:blue;font-weight: bold;">Exercise:</span> Apply the circular window and re-calculate the centroid of your noisy image

$$\bar{x} = \frac{\sum_{xy} x I_{xy} w_{xy}}{\sum_{xy} I_{xy} w_{xy}},~~ \bar{y} = \frac{\sum_{xy} y I_{xy} w_{xy}}{\sum_{xy} I_{xy} w_{xy}}.$$

Did it help? Could you imagine an iterative approach that would converge on a more correct centroid?

In [ ]:
# Your code here...




## Excercise 2: Fast Moving Star

*(Adapted from Gary Bernstein)*

This exercise uses two CCD images and the associated catalogs from the Dark Energy Survey (DES):
* First Image: [D00387788_r_c24_r3567p02_immasked.fits.fz](https://data.darkenergysurvey.org/fnalmisc/D00387788_r_c24_r3567p02_immasked.fits.fz)
* First Catalog: [D00387788_r_c24_r3567p02_red-fullcat.fits](https://data.darkenergysurvey.org/fnalmisc/D00387788_r_c24_r3567p02_red-fullcat.fits)
* Second Image: [D00704796_r_c35_r3518p01_immasked.fits.fz](https://data.darkenergysurvey.org/fnalmisc/D00704796_r_c35_r3518p01_immasked.fits.fz)
* Second Catalog: [D00704796_r_c35_r3518p01_red-fullcat.fits](https://data.darkenergysurvey.org/fnalmisc/D00704796_r_c35_r3518p01_red-fullcat.fits)

According to Gaia, there is a fast-moving star located at roughly (29.91148,-8.212267). If you display these two images using DS9, then align them using the Frame->Match->Frame->WCS option, you will be able to move your cursor to those coordinates actually see the star move.

We are going to practice using the WCS by starting from the windowed centroids of the catalog detections of this star, applying the WCS to transform the centroids to sky coordinates, and calculating the proper motion of the star.

In [ ]:
# Common astropy imports
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
# Open the FITS files. This returns HDU lists.

img1 = fits.open('D00387788_r_c24_r3567p02_immasked.fits.fz')
cat1 = fits.open('D00387788_r_c24_r3567p02_red-fullcat.fits')

img2 = fits.open('D00704796_r_c35_r3518p01_immasked.fits.fz')
cat2 = fits.open('D00704796_r_c35_r3518p01_red-fullcat.fits')

# Print the contents of the files
print("Image file:")
[print(hdu) for hdu in img1] 
print("Catalog file:")
[print(hdu) for hdu in cat1] 

In [ ]:
# You can access the header, image, and catalog data as shown below...
header = img1[1].header
image = img1[1].data
catalog = cat1[2].data

<span><span style="color:blue;font-weight: bold;">Exercise:</span> For each of the two images, get the Modified Julien Date (`MJD-OBS`) from the header and use the header to create a World Coordinate System (WCS). How far separated in time are the two images? How far separated in angle on the sky are the centers of the two CCDs?

In [ ]:
# Your code here...




<span><span style="color:blue;font-weight: bold;">Exercise:</span> Use the associated WCS to convert the windowed centroids of the catalogs into sky coordinates (RA, Dec). Find the catalog object associated with our target star at RA, DEC ~ (29.91148,-8.212267). What the the windowed centroid coordinates (`XWIN_IMAGE`, `YWIN_IMAGE`) for the detection? What is the major axis of the windowed image centroid error ellipse (`ERRAWIN_IMAGE`) that we will use as an estimate of the statistical uncertainty on the centroid?

In [ ]:
# Your code here...




<span><span style="color:blue;font-weight: bold;">Exercise:</span> Using the two sky coordinates calculated above to estimate the proper motion (in mas/yr) of the star between the two exposures. Note that proper motions are traditionally given in real angular motions, so that (RA PM) = (difference in RA) * cos(dec) / (time interval). Estimate the uncertainty on these measurements using the `ERRAWIN_IMAGE` estimates and a 10 mas turbulence uncertainty (added in quadrature).

Compare your measurement to the measurement from Gaia DR2:
```
pmra  =  326.06  +/- 0.94 mas/yr
pmdec = -124.12 +/- 0.81 mas/yr
```

Can you speculate about some inaccuracies in your estimate?

In [ ]:
# Your code here...


